In [4]:
import os
import re
from typing import Iterable, List, Optional


def remove_comments_from_code(code: str, extension: str) -> str:
    """Remove comments from code depending on file type."""
    # Java / Kotlin / C / C++ / JS / JSX / Gradle / TS / TSX
    if extension in ('.java', '.kt', '.js', '.jsx', '.c', '.cpp', '.h', '.gradle', '.ts', '.tsx'):
        pattern = r'(?m)^\s*//.*$|/\*[\s\S]*?\*/'
        return re.sub(pattern, '', code)

    # Python
    elif extension == '.py':
        pattern = r'(?m)^\s*#.*$|"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\''
        return re.sub(pattern, '', code)

    # XML / HTML
    elif extension in ('.xml', '.html'):
        pattern = r'<!--[\s\S]*?-->'
        return re.sub(pattern, '', code)

    # JSON / TXT / CSS
    else:
        return code


def print_folder_structure(root_dir: str, ignore_dirs: Optional[Iterable[str]] = None):
    """Print folder + file tree only."""
    ignore_dirs = set(ignore_dirs or [])

    for root, dirs, files in os.walk(root_dir):
        dirs[:] = [d for d in dirs if d not in ignore_dirs]

        depth = root.replace(root_dir, '').count(os.sep)
        indent = '│   ' * depth
        print(f"{indent}├── {os.path.basename(root) or root_dir}")

        sub_indent = '│   ' * (depth + 1)
        for file in files:
            print(f"{sub_indent}├── {file}")


def print_files_without_comments(
    root_dir: str,
    extensions: Optional[Iterable[str]] = None,
    print_structure_only: bool = False,
    ignore_dirs: Optional[List[str]] = None,
):
    """
    Print source files without comments OR only folder structure.
    """

    if extensions is None:
        extensions = (
            '.kt', '.java', '.xml', '.gradle',
            '.py', '.json', '.txt',
            '.js', '.jsx', '.ts', '.tsx', '.css', '.html'
        )

    ignore_dirs = set(ignore_dirs or [])

    if print_structure_only:
        print_folder_structure(root_dir, ignore_dirs)
        return

    for root, dirs, files in os.walk(root_dir):
        dirs[:] = [d for d in dirs if d not in ignore_dirs]

        for file in files:
            ext = os.path.splitext(file)[1]
            if ext in extensions:
                file_path = os.path.join(root, file)
                print(f"\n--- {file_path} ---")

                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        code = f.read()

                    cleaned_code = remove_comments_from_code(code, ext)

                    for line in cleaned_code.splitlines():
                        if line.strip():
                            print(line)

                except Exception as e:
                    print(f"[Error reading {file_path}: {e}]")


# =========================
# Example Usage
# =========================

directory_path = r"src/components/mindmap"

print_files_without_comments(
    root_dir=directory_path,
    print_structure_only=False,
    ignore_dirs=["assets", ".git", "dist", "__pycache__"]
)


--- src/components/mindmap\ControlNode.tsx ---
import React, { useState } from "react";
import {
	Activity,
	Zap,
	AlertTriangle,
	Settings,
	ChevronDown,
	Info,
	Sliders,
} from "lucide-react";
import {
	PropertyGrid,
	ActionRow,
	ToggleRow,
	Sparkline,
} from "./ControlNodeWidgets";
export type NodeType = "EQUIPMENT" | "LOGIC" | "SENSOR" | "PID";
export interface NodeMetadata {
	tagId?: string;
	narrativeRef?: string;
	unit?: string;
	minValue?: number;
	maxValue?: number;
	properties?: { label: string; value: string | number }[]; // Replaces fixed setpoint/capacity
	actions?: {
		label: string;
		type: "primary" | "danger" | "neutral";
		icon?: "play" | "stop" | "reset" | "ack";
	}[];
	toggles?: { label: string; checked: boolean }[];
	trendData?: number[]; // Array of numbers for Sparkline
}
export interface ControlNodeData {
	id: string;
	type: NodeType;
	label: string;
	status: "active" | "inactive" | "fault";
	meta: NodeMetadata;
	currentValue?: number;
}
const NodeIcon = ({ typ